![](https://upload.wikimedia.org/wikipedia/commons/thumb/9/9b/Carl_Friedrich_Gauss.jpg/330px-Carl_Friedrich_Gauss.jpg)

# Описание метода

Даны матрица $A$ размером $N\times N$, вектор $B$ высотой $N$. Требуется найти вектор $X$ такой, что $AX=B$.

Без терминологии линейной алгебры это записывается, как:

$
\begin{array}{lcr}
a_{11}x_1+\ldots+a_{1N}x_N &=& b_1 \\
\ldots & & \\
a_{N1}x_1+\ldots+a_{NN}x_N &=& b_N \\
\end{array}
$

## Прямой ход

Из второго уравнения вычитаем первое, умноженное на $\frac{a_{21}}{a_{11}}$. Из третьего вычитаем первое, умноженное на $\frac{a_{31}}{a_{11}}$. И т.д. до $N$-го. Получаем систему уравнений следующего вида.

$
\begin{array}{lcr}
a_{11}&x_1 +& a_{12} x_2 +\ldots+a_{1N}x_N &=& b_1 \\
0 &x_1 +& a'_{22} x_2 +\ldots+a'_{1N}x_N &=& b'_1 \\
& \ldots & & \\
0 &x_1 +& a'_{N2} x_2 +\ldots+a'_{NN}x_N &=& b'_N \\
\end{array}
$

Затем из третьего уравнения вычитаем второе, умноженное на $\frac{a_{32}}{a_{22}}$.

И т.д., пока не выполним такое же действие над последней строкой.

В итоге мы получим систему уравнений следующего вида.

$
\begin{array}{lcr}
a_{11}& x_1 +& a_{12}   & x_2 + & a''_{13} & x_3 + \ldots + a_{1N}   x_N &=& b_1 \\
0     & x_1 +& a''_{22} & x_2 + & a''_{23} & x_3 + \ldots + a''_{2N} x_N &=& b''_1 \\
0     & x_1 +& 0        & x_2 + & a''_{33} & x_3  +\ldots + a''_{3N} x_N &=& b''_3 \\
0     & x_1 +& 0        & x_2 + & 0        & x_3 + \ldots + a''_{4N} x_N &=& b''_4 \\
& & & \ldots & & & & \\
0 & x_1 +& & \ldots & +                  & 0 x_{N-1} + a''_{NN} x_N &=& b''_N \\
\end{array}
$


## Обратный ход

Очевидно, $x_N = \frac{b''_N}{a''_{NN}}$.

Далее, $x_i = \frac{b''_i - \sum_{j=i+1}^{N} a''_{ij}x_j}{a_{ii}}$


# Задание

Предлагается реализовать слкдующие недостающие функции

In [1]:
import copy  # to copy list of lists

def gauss(a, b):
    a = copy.deepcopy(a)  # to copy internal lists too
    b = b.copy()

    def forward():
        # do something to a and b
        pass

    def backward():
        x = [0] * len(b)
        # then do something to a and b
        pass
        return x

    forward()
    return backward()

a = [
    [1,2,1],
    [3,2,3],
    [1,0,0]
]

b = [5,6,7]

print(gauss(a, b))


[0, 0, 0]


Желающие вместо встроенных списков сразу могут использовать `numpy.array` — всё равно этим кончится.

In [2]:
import numpy

# ...
# ...

a = numpy.array([
    [1,2,1],
    [3,2,3],
    [1,0,0]
])

b = numpy.array([5,6,7])

# Somewhere in reverse
# ...
# x = numpy.zeros(len(b))
# ...

print(gauss(a, b))

[0, 0, 0]
